# Streaming Graphs

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

import graphcat.notebook
import numpy

In [2]:
graph = graphcat.StreamingGraph()
logger = graphcat.Logger(graph)
graph.set_task("a", graphcat.array(numpy.arange(16)))

In [3]:
graph.output("a", extent=graphcat.ArraySlice[0:4])

INFO:graphcat.common:Task a updating.
INFO:graphcat.common:Task a executing. Inputs: {} Extent: slice(0, 4, None)
INFO:graphcat.common:Task a finished. Output: [0 1 2 3]


array([0, 1, 2, 3])

In [4]:
graph.output("a", extent=graphcat.ArraySlice[0:4])

INFO:graphcat.common:Task a updating.


array([0, 1, 2, 3])

In [5]:
graph.output("a")

INFO:graphcat.common:Task a updating.
INFO:graphcat.common:Task a executing. Inputs: {} Extent: None
INFO:graphcat.common:Task a finished. Output: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [6]:
graph.output("a")

INFO:graphcat.common:Task a updating.


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [7]:
graph.output("a", extent=graphcat.ArraySlice[::2])

INFO:graphcat.common:Task a updating.
INFO:graphcat.common:Task a executing. Inputs: {} Extent: slice(None, None, 2)
INFO:graphcat.common:Task a finished. Output: [ 0  2  4  6  8 10 12 14]


array([ 0,  2,  4,  6,  8, 10, 12, 14])

In [8]:
graph.output("a", extent=graphcat.ArraySlice[[1, 2, 3]])

INFO:graphcat.common:Task a updating.
INFO:graphcat.common:Task a executing. Inputs: {} Extent: [1, 2, 3]
INFO:graphcat.common:Task a finished. Output: [1 2 3]


array([1, 2, 3])